In [1]:
import sys
import os
import pathlib
import glob
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
from tqdm.auto import tqdm
from dateutil import parser
# from datetime import datetime, timedelta

import matplotlib.pyplot as plt
# import seaborn as sns
# import plotly.graph_objs as go
# from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [65]:
X = torch.rand([4, 48, 5]) # [batch_size, time_steps, num_features]

batch_size = X.size(0)

C = X.unsqueeze(1) # [batch_size, num_channels=1, time_steps, num_features]
C = F.relu(
    nn.Conv2d(1, 32,kernel_size=(7, 5))(C)
    ) # [batch_size, conv1_out_channels, shrinked_time_steps, 1]
C = nn.Dropout(p = 0.2)(C)
C = torch.squeeze(C, 3) # [batch_size, conv1_out_channels, shrinked_time_steps]

C.size()

torch.Size([4, 32, 42])

In [66]:
shrinked_time_steps = C.size(2)
shrinked_time_steps

42

In [67]:
R = C.permute(0, 2, 1) # [batch_size, shrinked_time_steps, conv1_out_channels]
out, hidden = nn.GRU(32, 64, batch_first=True)(R) # [batch_size, shrinked_time_steps, recc_out_channels]
R = out[:, -1, :] # [batch_size, recc_out_channels]
R = nn.Dropout(p = 0.2)(R)

R.size()

torch.Size([4, 64])

In [68]:
skip_reccs_out_channels = [4,4]
skip_steps = [4,24]

skip_reccs = {}
for i, step in enumerate(skip_steps):
    skip_reccs[i] = nn.GRU(32, skip_reccs_out_channels[i], batch_first=True)
    
skip_reccs

{0: GRU(32, 4, batch_first=True), 1: GRU(32, 4, batch_first=True)}

In [69]:
for i, step in enumerate(skip_steps):
    # print(i,step)
    skip_step = step
    skip_sequence_len = shrinked_time_steps // skip_step # self.pt
    print(shrinked_time_steps, skip_step, skip_sequence_len)
    
    # shrinked_time_steps shrinked further
    S = C[:, :, -skip_sequence_len*skip_step:] # [batch_size, conv1_out_channels, shrinked_time_steps]
    S = S.view(S.size(0), S.size(1), skip_sequence_len, skip_step) # [batch_size, conv1_out_channels, skip_sequence_len, skip_step=num_skip_components]
    # note that num_skip_components = skip_step
    S = S.permute(0, 3, 2, 1).contiguous() # [batch_size, skip_step=num_skip_components, skip_sequence_len, conv1_out_channels]
    S = S.view(S.size(0)*S.size(1), S.size(2), S.size(3))  # [batch_size*num_skip_components, skip_sequence_len, conv1_out_channels]
    out, hidden = skip_reccs[i](S) # [batch_size*num_skip_components, skip_sequence_len, skip_reccs_out_channels[i]]
    S = out[:, -1, :] # [batch_size*num_skip_components, skip_reccs_out_channels[i]]
    S = S.view(batch_size, skip_step*S.size(1)) # [batch_size, num_skip_components*skip_reccs_out_channels[i]]
    S = nn.Dropout(p = 0.2)(S)
    R = torch.cat((R, S), 1) # [batch_size, recc_out_channels + skip_reccs_out_channels * num_skip_components]
    
    print(R.size())

42 4 10
torch.Size([4, 80])
42 24 1
torch.Size([4, 176])


In [70]:
# Output Layer
O = F.relu(nn.Linear(
    64 + np.dot(skip_steps, skip_reccs_out_channels),
    1)(R)) # [batch_size, output_out_features=1]
O.size()

torch.Size([4, 1])

In [71]:
AR = X[:, -7:, 3:4] # [batch_size, ar_window_size, output_out_features=1]
AR = AR.permute(0, 2, 1).contiguous() # [batch_size, output_out_features, ar_window_size]
AR = nn.Linear(7, 1)(AR) # [batch_size, output_out_features, 1]
AR = AR.squeeze(2) # [batch_size, output_out_features]
O = O + AR

print(AR.size())
print(O.size())

torch.Size([4, 1])
torch.Size([4, 1])


In [72]:
O

tensor([[-0.4087],
        [-0.2802],
        [-0.1295],
        [-0.0511]], grad_fn=<AddBackward0>)

In [3]:
%load_ext autoreload
%autoreload 2

from helper_functions import epa_taiwan_data_pipeline
from models import lstnet_gokul, lstnet_laigoukun

In [4]:
device = "cuda" if torch.cuda.is_available else "cpu"
device

'cuda'

In [5]:
root_dir = pathlib.Path(os.getcwd()).parent
raw_data_dir = root_dir / "data/0_raw"
processed_data_dir = root_dir / "data/1_processed"
experiment_dir = root_dir / "experiment"

# Data Loading and Preprocessing

- Import the data
- Feature engineering
- Turn the data into tensor

## Import the data

In [6]:
year = 2018
site_name = "Banqiao"
columns = ["SiteEngName","PM2.5","AMB_TEMP","CH4",'CO',"NMHC","read_time"]

# import data
pm25_df = epa_taiwan_data_pipeline.import_epa_data(site_name=site_name, year=year)[columns]

# basic preprocessing
pm25_df = epa_taiwan_data_pipeline.standardize_df(pm25_df)

## Feature engineering

In [7]:
train_split = 0.6

train_data = pm25_df.iloc[:int(len(pm25_df)*train_split),:]
print(f"All data length:{len(pm25_df)} \nTrain data length:{len(train_data)}")
train_data.tail()

All data length:8760 
Train data length:5256


,siteengname,pm2.5,amb_temp,ch4,co,nmhc,read_time
399077,Banqiao,11.0,31.4,1.8,0.41,0.13,2018-08-07 19:00:00
399153,Banqiao,11.0,30.8,1.8,0.35,0.10,2018-08-07 20:00:00
399229,Banqiao,10.0,30.4,1.8,0.31,0.09,2018-08-07 21:00:00
399305,Banqiao,8.0,30.2,1.8,0.30,0.08,2018-08-07 22:00:00
399381,Banqiao,9.0,29.9,1.8,0.23,0.06,2018-08-07 23:00:00


In [8]:
normalized_columns = ['pm2.5', 'amb_temp', 'ch4', 'co', 'nmhc']

for column in normalized_columns:
    normalized_column_name = column + '_normalized'
    train_data[normalized_column_name] = (train_data[column] - train_data[column].min()) / (train_data[column].max() - train_data[column].min())

## Convert to tensor

In [9]:
class AqiDataset(Dataset):
    def __init__(self, data, history_len):
        self.data = data
        self.history_len = history_len
        
    def __len__(self):
        self.len = len(self.data) - self.history_len  
        return self.len
    
    def __getitem__(self, index):
        x_cols = ['pm2.5_normalized', 'amb_temp_normalized', 'ch4_normalized', 'co_normalized', 'nmhc_normalized']
        y_cols = ['pm2.5_normalized']
        x = self.data.iloc[index: index+self.history_len, :][x_cols].values
        y = self.data.iloc[index+self.history_len, :][y_cols].values.astype('float')
        x = torch.tensor(x).float()
        y = torch.tensor(y).float()
        return x, y

In [10]:
# verify dataset instances
temp_train_dataset = AqiDataset(train_data, history_len=48)
print(len(temp_train_dataset))
x, y = temp_train_dataset[0]
print(x.shape, y.shape)

5208
torch.Size([48, 5]) torch.Size([1])


In [11]:
# train data_loader
temp_train_data_loader = DataLoader(temp_train_dataset, batch_size=4)
X, Y = next(iter(temp_train_data_loader))
print(X.shape, Y.shape)

torch.Size([4, 48, 5]) torch.Size([4, 1])


# Training Pipeline

- Prepare the model
- Initiate loss and optimization function
- Training process
- Plot the loss curve

In [12]:
# test the model
temp_model = lstnet_gokul.LSTNet()

for X, Y in temp_train_data_loader:
    print(X.shape)
    out = temp_model(X)
    print(Y.shape, out.shape)
    break

torch.Size([4, 48, 5])
torch.Size([4, 1]) torch.Size([4, 1])


In [13]:
history_len = 48
batch_size = 8 

epochs = 10

lr = 0.01
weight_decay = 0.01

# Experimentation

- Prepare different scenarios
    - [x] number of epochs --> [50, 100, 200]
    - [x] lookback periods --> [24, 24x2, 24x7, 24x30] (history_len)
    - [x] batch size --> [16, 64, 128]
    - [x] loss function --> [MSE (nn.MSELoss()), MAE (nn.L1Loss()), Huber Loss (nn.SmoothL1Loss())]
- Log the experiment
- Monitor the result with MLFlow or ~~tensorboard~~

In [14]:
epochs = [10, 20, 50]
lookback_periods = [24//2, 24, 24*2, 24*7]
batch_sizes = [16, 64, 128]
loss_functions = [nn.MSELoss(), nn.SmoothL1Loss()]

# Inferencing

- Load the best model
- Prepare the test data
- Save the result

# Deployment